In [1]:
import datetime
import json
import pandas as pd
import openpyxl

In [2]:
# https://docs.google.com/spreadsheets/d/1Ul8iMTsOFUKUmqz6MK0zpgt8Ki8tFtoWKGlwXj-Op34/edit?usp=sharing
XLS_PATH = 'data/Optimism GovFund Grants_ Public Delivery Tracking.xlsx'
WB = openpyxl.load_workbook(XLS_PATH)
sheets = [x for x in WB.sheetnames if x != 'Status Key']

In [8]:
def convert_datetime(obj):
    if isinstance(obj, datetime.datetime):
        return obj.isoformat()
    elif isinstance(obj, list):
        return [convert_datetime(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_datetime(value) for key, value in obj.items()}
    return obj

def load_sheet(sheet_name):
    
    sheet = WB[sheet_name]
    
    for skiprows in range(0,2):
        df = pd.read_excel(XLS_PATH, sheet_name=sheet_name, skiprows=skiprows, thousands=',')
        if 'Status' in df.columns:
            break
            
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            print(col)
            df[col] = df[col].astype(str)
        
    this_cycle = None
    cols = [
        'Status', 'Cycle #', 'Total Amount (OP)', 'Amount (OP)',
        'Project Name', 'Proposal Link', 'L2 Address'
    ]
    data = []
    for row_num, (_, row) in enumerate(df.iterrows()):

        if isinstance(row['Cycle #'], str):
            this_cycle = row['Cycle #']
        
        status = row.get('Status')
        if not isinstance(status, str):
            continue
        if status in ('Not-passed', 'Rejected'):
            continue


        amount = row.get('Total Amount (OP)', row.get('Amount (OP)'))
        amount_comment = None
        try:
            amount = pd.to_numeric(amount)
        except:
            amount_comment = amount
            amount = None    
            
        project_name = row['Project Name']
        proposal_link = row['Proposal Link']
        if not isinstance(proposal_link, str):
            cell = sheet.cell(skiprows+row_num+2, 2)
            try:
                proposal_link = cell.hyperlink.target
            except:
                print("No link for project:", project_name, cell)
                proposal_link = ""
        wallet_address = row['L2 Address']
        wallet_address = wallet_address.lower() if isinstance(wallet_address, str) else ""
        other_fields = row[[x for x in row.keys() if x not in cols]].dropna().apply(convert_datetime).to_dict()
        data.append({
            'round': sheet_name,
            'cycle': this_cycle,
            'project_name': project_name,
            'status': status,
            'proposal_link': proposal_link,
            'wallet_address': wallet_address,
            'amount': amount,
            'amount_comment': amount_comment,
            'meta': other_fields
        })

    return data
    
json_data = []
for s in sheets:
    json_data.extend(load_sheet(s))

No link for project: Majora Security Grant Application <Cell 'Grants Season 6'.B119>
No link for project: Majora Security Grant Application <Cell 'Grants Season 6'.B120>


In [9]:
with open("govgrants.json", "w") as f:
    json.dump(json_data, f, indent=2)